In [6]:
# %pip install sqlalchemy En caso de ser necesario, ejecutar
# %pip install --user psycopg2 En caso de ser necesario, ejecutar
# %pip install numpy En caso de ser necesario, ejecutar
# %pip install pandas En caso de ser necesario, ejecutar

In [7]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [8]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

In [9]:
novedad_servicio = pd.read_sql_table('mensajeria_novedadesservicio', src)
dim_tipo_novedad = pd.read_sql_table('dim_tipo_novedad', etl)
hecho_novedad = novedad_servicio.groupby(['servicio_id', 'tipo_novedad_id']).size().reset_index(name='total_novedad')

hecho_novedad.rename(columns={
    'servicio_id': 'key_servicio',
    'tipo_novedad_id': 'key_tipo_novedad',
}, inplace=True)

hecho_novedad["key_hecho_novedad"] = range(1, len(hecho_novedad) + 1)

# Mover la columna 'telefono' al final
cols = [col for col in hecho_novedad.columns if col != 'total_novedad'] + ['total_novedad']
hecho_novedad = hecho_novedad[cols]


# Eliminar filas duplicadas en 'hecho_novedad'
hecho_novedad = hecho_novedad.drop_duplicates()

hecho_novedad

,key_servicio,key_tipo_novedad,key_hecho_novedad,total_novedad
0,29,1,1,1
1,29,2,2,1
2,51,1,3,5
3,51,2,4,3
4,57,1,5,2
...,...,...,...,...
3557,28455,1,3558,1
3558,28461,2,3559,1
3559,28464,2,3560,1
3560,28466,2,3561,1


In [10]:
hecho_novedad.to_sql('hecho_novedad', etl, index=False, if_exists='replace')

562